In [6]:
import tensorflow as tf
import numpy as np

In [2]:
class CustomDense(tf.keras.layers.Layer):
    def __init__(self, units=32):
        super(CustomDense, self).__init__()
        self.units = units
    
    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            name='kernel', 
            initial_value=w_init(
                shape=(input_shape[-1], self.units), dtype=tf.float32
            ),
            trainable=True
        )
        
        # Initialoze the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            name='bias',
            initial_value=b_init(
                shape=(self.units,), dtype='float32'
            ),
            trainable=True
        )
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [4]:
# declare an instance of the class
my_dense = CustomDense(units=1)

# define an input and feed into the layer
x = tf.ones((1, 1))
y = my_dense(x)

# Parameters of the base layer class like `variables` can be used
print(my_dense.variables)

[<tf.Variable 'custom_dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.04425662]], dtype=float32)>, <tf.Variable 'custom_dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [7]:
x_train = np.array([1., 2., 3., 4., 5.])
y_train = np.array([1., 4., 9., 16., 25.])

inp = tf.keras.Input((1,))
x = CustomDense(128)(inp)
x = CustomDense(128)(x)
x = CustomDense(128)(x)
output = tf.keras.layers.Dense(1)(x)



In [8]:
model = tf.keras.Model(inputs=inp, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
custom_dense_1 (CustomDense) (None, 128)               256       
_________________________________________________________________
custom_dense_2 (CustomDense) (None, 128)               16512     
_________________________________________________________________
custom_dense_3 (CustomDense) (None, 128)               16512     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 33,409
Trainable params: 33,409
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam', loss='mae', metrics=['accuracy'])
hisory = model.fit(x_train, y_train, epochs=500, verbose=0)

In [10]:
model.predict([6.])

array([[28.646996]], dtype=float32)